# MongoDB

MongoDB is a no-sql database for storage of JSON-style data (corresponding to dictionaries and lists in Python).

https://docs.mongodb.com/manual/tutorial/getting-started/

In contrast to relational databases, MongoDB documents (corresponding to table entries in relational databases) in collections (corresponding to tables) have no fixed structure. However, it makes sense to define all documents in one collection as similar as possible to make queries, etc. on them easier.

MongoDB works both on single machines and distributed along multiple clusters. Here, distribution is however not discussed.

This tutorial requires a running MongoDB server, preferably in a Docker container started using the given docker-compose file.

In [2]:
!conda install pymongo mongoengine --yes # installs low-level and high-level APIs

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - mongoengine
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    mongoengine-0.18.2         |           py37_0         157 KB  conda-forge
    pymongo-3.8.0              |   py37he1b5a44_0         966 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following NEW packages will be INSTALLED:

  mongoengine        conda-forge/linux-64::mongoengine-0.18.2-py37_0
  pymongo            conda-forge/linux-64::pymongo-3.8.0-py37he1b5a44_0



mongoengine-0.18.2   | 157 KB    | ##################################### | 100% 
pymongo-3.8.0        | 966 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transact

In [2]:
import pymongo
import gridfs
import mimetypes

## Connecting

In [33]:
url = 'mongo_db' # Docker alias
port = 27017
db_name = 'tutorial_db'
con_str = f'mongodb://{url}:{port}'
client = pymongo.MongoClient(con_str)
db = client[db_name] # create a new database or open existing database

In [37]:
db.list_collection_names() # should be empty for new database

[]

## Storing and Retreiving Documents

In [13]:
docs = db.docs

In [14]:
entry = {'name': 'file1',
        'tags': ['tax', 'bill'],
        'Status': 'payed'}
result = docs.insert_one(entry)

In [15]:
result.inserted_id

ObjectId('5d24e6ff8e16a0e4fa656ca9')

In [16]:
docs.find_one()

{'_id': ObjectId('5d24e6ff8e16a0e4fa656ca9'),
 'name': 'file1',
 'tags': ['tax', 'bill'],
 'Status': 'payed'}

## GridFS

An API to store and retreive files of any size from a MongoDB.

https://docs.mongodb.com/manual/core/gridfs/

In [17]:
fs = gridfs.GridFS(db)

### Store files in db

In [18]:
filename = '../code/test_data.csv' # uses test data from Pandas tutorial
metadata = {'description': 'NYC taxi data',
            'tags': ['test', 'large'],
            }
content_type = mimetypes.guess_type(filename)[0] # automatically infer data type
content_type

'text/csv'

In [19]:
with open(filename,'rb') as f:
    fs.put(f, filename=filename, metadata=metadata, content_type=content_type)

### Check db content

In [21]:
fs.list() # list of all stored files

['../code/test_data.csv']

In [22]:
db.list_collection_names() # note the 2 documents for fs

['docs', 'fs.files', 'fs.chunks']

In [20]:
list(db.fs.files.find()) # storage of file metadata

[{'_id': ObjectId('5d24e7848e16a0e4fa656caa'),
  'filename': '../code/test_data.csv',
  'metadata': {'description': 'NYC taxi data', 'tags': ['test', 'large']},
  'contentType': 'text/csv',
  'md5': '8c2bdc4b7a49464147d557568a2979bf',
  'chunkSize': 261120,
  'length': 70930317,
  'uploadDate': datetime.datetime(2019, 7, 9, 19, 14, 15, 263000)}]

In [29]:
db.fs.chunks.count_documents({}) # here the actual data is stored in chunks

272

### Get Files from db

In [30]:
with open('test_output.csv','wb') as f:
    f.write(fs.get_last_version(filename).read())

### Delete all Files

In [31]:
for f in fs.find():
    print(f._id)
    fs.delete(f._id)

5d24e7848e16a0e4fa656caa


In [32]:
%rm test_output.csv

# MongoEngine

MongoEngine is a Document-Object Mapper (think ORM, but for document databases) for working with MongoDB from Python.

It allows to define a (flexible) schema and validation rules for documents,

http://mongoengine.org/

http://docs.mongoengine.org/

In [38]:
import mongoengine

## Connecting

In [39]:
mongoengine.connect(db=db_name, host=url, port=port)

MongoClient(host=['mongo_db:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## Defining Document Schema

In [40]:
class Doc(mongoengine.Document):
    name = mongoengine.StringField(required=True)
    status = mongoengine.StringField(choices=['open', 'done'])
    tags = mongoengine.ListField(mongoengine.StringField())

## Storing Data

In [41]:
doc1 = Doc(name='bill1', status='open', tags=['tax'])
doc2 = Doc(name='bill2', status='done', tags=['tax', 'handyman'])

In [42]:
doc1.save()
doc2.save()

<Doc: Doc object>

In [46]:
doc3 = Doc(name='invalid_bill', status='invalid', tags=['tax', 'handyman'])
try:
    doc3.save()
except mongoengine.ValidationError as e:
    print(e)

ValidationError (Doc:None) (Value must be one of ['open', 'done']: ['status'])


## Retreiving Data

In [47]:
for doc in Doc.objects(tags='handyman'):
    print(doc.name)

bill2


# Cleanup

In [36]:
client.drop_database(db_name)